# <center>CryptoCurrencies CA#2<center>
<h1><center>Transacting on Bitcoin TestNet / Firing up an Ethereum Node<center></h1>

#### <center>"Amirali - Rahimi"<center>
#### <center>"810100146"<center>

### 1. Address Generation:

 1.1: Base58 and WIF address

In [11]:
import os
import ecdsa
import hashlib
import base58

In [12]:
def private_key_to_wif(private_key):
    extended_key = b'\xEF' + private_key #0xEF for testnet
    sha256_1 = hashlib.sha256(extended_key).digest()
    sha256_2 = hashlib.sha256(sha256_1).digest()
    
    checksum = sha256_2[:4]
    extended_key_with_checksum = extended_key + checksum
    wif = base58.b58encode(extended_key_with_checksum)
    return wif

In [13]:
def generate_private_key_and_bitcoin_address():
    private_key = os.urandom(32)

    sk = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)
    vk = sk.verifying_key
    public_key = b'\x04' + vk.to_string()

    sha256_pk = hashlib.sha256(public_key).digest()
    ripemd160 = hashlib.new('ripemd160')
    ripemd160.update(sha256_pk)
    ripemd160_pk = ripemd160.digest()
    network_byte = b'\x6F' + ripemd160_pk #0x6F for testnet
    sha256_nb = hashlib.sha256(network_byte).digest()
    sha256_nb2 = hashlib.sha256(sha256_nb).digest()
    checksum = sha256_nb2[:4]
    binary_address = network_byte + checksum
    testnet_address = base58.b58encode(binary_address)

    return private_key, public_key, testnet_address

In [4]:
def printThese(private_key, public_key, testnet_address):
    print("Private Key (hex):", private_key.hex())
    print("Public Key (hex):", public_key.hex())
    print("Testnet Bitcoin Address:", testnet_address.decode())
    private_key_wif = private_key_to_wif(private_key)
    print("Private Key (wif):", private_key_wif.decode())

In [5]:
private_key, public_key, testnet_address = generate_private_key_and_bitcoin_address()
printThese(private_key, public_key, testnet_address)

Private Key (hex): 81898e39441946ea1f10a66263d12a45da91bfd203206ad4f787d446fd91ba4a
Public Key (hex): 042cd341a5b77f678f2af02df1a46d979f8a1e208b94a74e2639159d2074dd14a15b94da5590ce5913b6d60b03c2712a33d72edb6958baf667a7f1e13052815291
Testnet Bitcoin Address: n1aAe6wkAPEqJ69wJrZLsRfbhqhGbGXWQe
Private Key (wif): 92Zy28i3Y1fGfq4VNfhhGukcYYm8TW6ro5pBrbadH36yco2jiZi


1.2: Vanity Address Generator


In [6]:
private_key, public_key, testnet_address = None, None, None
vanity_address_start = "oon"
while(True):
    private_key, public_key, testnet_address = generate_private_key_and_bitcoin_address()
    if(testnet_address.decode()[1:4]==vanity_address_start):
        break

printThese(private_key, public_key, testnet_address)

Private Key (hex): 6c2f522f647d774d34ba05f6bc2f4eed53fcbda5bad13499099d2b8b64976cbe
Public Key (hex): 045a85a920b66312907ea8e2b716d1cf6ef95b4f98af0640964a1f4da46596abde167ceb877dd74188163955323791a6fc60c7a2393158163d605aa22fd79b9c5f
Testnet Bitcoin Address: moonTrsMrR1D8YKUbkbKaZXapBa4nKYSSF
Private Key (wif): 92QZbkTMzm1SEDz63sJtRKkryn8wBouHPhweFJbQ4W2t1qhkBfz


### 2. Transacting on Bitcoin TestNet:

Here are some utility functions that you will need to use in order to make transactions. Fill the #TODO parts in the functions in order to be used in the next steps:

#### Utility Functions:

In [14]:
import requests
from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH

def create_txin(txid, utxo_index):
    return CMutableTxIn(COutPoint(lx(txid), utxo_index))


def create_txout(amount, scriptPubKey):
    return CMutableTxOut(amount*COIN, CScript(scriptPubKey))


def create_OP_CHECKSIG_signature(txin, txouts, txin_scriptPubKey, seckey):
    tx = CMutableTransaction([txin], txouts)
    sighash = SignatureHash(CScript(txin_scriptPubKey), tx, 0, SIGHASH_ALL)
    sig = seckey.sign(sighash) + bytes([SIGHASH_ALL])
    return sig


def create_signed_transaction(txin, txouts, txin_scriptPubKey, txin_scriptSig):
    tx = CMutableTransaction([txin], txouts)
    txin.scriptSig = CScript(txin_scriptSig)
    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))
    return tx


def broadcast_transaction(tx):
    raw_transaction = b2x(tx.serialize())
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    return requests.post(
        'https://api.blockcypher.com/v1/btc/test3/txs/push',
        headers=headers,
        data='{"tx": "%s"}' % raw_transaction,
    )

In [15]:
import bitcoin.wallet
from bitcoin.core import COIN, b2lx, serialize, x, lx, b2x

bitcoin.SelectParams("testnet")

def P2PKH_scriptPubKey(address):
    return [OP_DUP, OP_HASH160, address, OP_EQUALVERIFY, OP_CHECKSIG]


def P2PKH_scriptSig(txin, txouts, txin_scriptPubKey, private_key, public_key):
    signature = create_OP_CHECKSIG_signature(txin, txouts, txin_scriptPubKey, private_key)
    return [signature, public_key]


def send_from_P2PKH_transactions(amounts, txid_to_spend, utxo_index, txout_scriptPubKeys, sender_address, private_key, public_key):
    txouts = [create_txout(amount, scriptPubKey) for amount, scriptPubKey in zip(amounts, txout_scriptPubKeys)]
    txin_scriptPubKey = P2PKH_scriptPubKey(sender_address)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = P2PKH_scriptSig(txin, txouts, txin_scriptPubKey, private_key, public_key)
    new_tx = create_signed_transaction(txin, txouts, txin_scriptPubKey, txin_scriptSig)
    return broadcast_transaction(new_tx)


Now use the provided functions to make these transactions:  
`All the transaction photos and links are available in the report`

2.1: One input and two outputs transaction

### repository address using here:
`Private Key (hex)`: 54011a9093821a6fcb220e078c6d57618d81824b42187c6cb89a377aff24580a  
`Public Key (hex)`:   0470841664631eebe9c1df5972786efef7ab35d5442afd20cf2a85df401c0cb4df8e29ced7be5e5690399b70f6d055ec1b7e6506cd43c67bda097b5a3335ef9127  
`Testnet Bitcoin Address`: moonuSvwZDL5HExuVQuL223RSw63fhJsaL  
`Private Key (wif)`: 92DuwWRXqqF3yxJCvSUvkcjPE2phy2XMS1jGaM8BQMkUfx7dnPk

coin recieved from faucet 1: https://live.blockcypher.com/btc-testnet/tx/7118e8be46e26d09b61263d96b6595fdf8cba651bdc581bde626cf6cc1b28691/

In [23]:
my_private_key = bitcoin.wallet.CBitcoinSecret('92DuwWRXqqF3yxJCvSUvkcjPE2phy2XMS1jGaM8BQMkUfx7dnPk')
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

amount_to_send = 0.00016811
txid_to_spend = '7118e8be46e26d09b61263d96b6595fdf8cba651bdc581bde626cf6cc1b28691'
utxo_index = 1

unspendable_scriptPubKey = [OP_RETURN]
anyone_can_spend_scriptPubKey = [OP_TRUE]
amounts = [0.00006811, 0.0001]

print("My Address:", my_address)
print("Public Key (hex):", my_public_key.hex())
print("Private Key (WIF):", my_private_key)
response = send_from_P2PKH_transactions(amounts, txid_to_spend, utxo_index, [unspendable_scriptPubKey, anyone_can_spend_scriptPubKey], my_address, my_private_key, my_public_key)
print("First transaction broadcasted")
print(response.status_code, response.reason)
print(response.text)

My Address: moonuSvwZDL5HExuVQuL223RSw63fhJsaL
Public Key (hex): 0470841664631eebe9c1df5972786efef7ab35d5442afd20cf2a85df401c0cb4df8e29ced7be5e5690399b70f6d055ec1b7e6506cd43c67bda097b5a3335ef9127
Private Key (WIF): 92DuwWRXqqF3yxJCvSUvkcjPE2phy2XMS1jGaM8BQMkUfx7dnPk
First transaction broadcasted
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "2f082c2def33a2b60b9835e6e178015895f1ab401619bee425b79d8b1698282a",
    "addresses": [
      "moonuSvwZDL5HExuVQuL223RSw63fhJsaL"
    ],
    "total": 16811,
    "fees": 0,
    "size": 209,
    "vsize": 209,
    "preference": "low",
    "relayed_by": "202.61.242.34",
    "received": "2024-05-30T15:01:29.46274603Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 2,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "7118e8be46e26d09b61263d96b6595fdf8cba651bdc581bde626cf6cc1b28691",
        "output_index": 1,
        "script": "47304402205ee338f843293450f8172f87bf2fec3cd5a

In above cell, we created a transaction with 2 output one of them spendable by anyone and the other one is unspendable one. Now we spend the spendable one.

In [24]:
txid_to_spend_2 = '2f082c2def33a2b60b9835e6e178015895f1ab401619bee425b79d8b1698282a'
utxo_index_2 = 1

txout_scriptPubKey_2 = P2PKH_scriptPubKey(my_address)

response_2 = send_from_P2PKH_transactions([0.0001], txid_to_spend_2, utxo_index_2, [txout_scriptPubKey_2], my_address, my_private_key, my_public_key)
print("Second transaction broadcasted")
print(response_2.status_code, response_2.reason)
print(response_2.text)

Second transaction broadcasted
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "f101e3d17cf5bdbfa1423d7d572e4be77d7e0ec7bd98cbbaa10fb95ad9bb6d40",
    "addresses": [
      "moonuSvwZDL5HExuVQuL223RSw63fhJsaL"
    ],
    "total": 10000,
    "fees": 0,
    "size": 223,
    "vsize": 223,
    "preference": "low",
    "relayed_by": "202.61.242.34",
    "received": "2024-05-30T15:06:52.404704971Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "2f082c2def33a2b60b9835e6e178015895f1ab401619bee425b79d8b1698282a",
        "output_index": 1,
        "script": "47304402202ea8250cd55c7d3b9d63259df8480c12194775275a4cdef290e1c3386e76f98c0220760697a20457cc7c43b4660ba22e28dfc95c0403999d493ee1a3f484937e057001410470841664631eebe9c1df5972786efef7ab35d5442afd20cf2a85df401c0cb4df8e29ced7be5e5690399b70f6d055ec1b7e6506cd43c67bda097b5a3335ef9127",
        "output_value": 10000,


## Address 1:
`Private Key (hex)`: 013da2dadc13390055ede81574ec9affc3c8d1e325a89456eb98e0ebee43e7eb  
`Public Key (hex)`: 0440a061c0a6575528f43e499dffe98471933296513ff3a816a91600c5a775b7c421b43e9945d3c420ed614915574284f4351bf715b8c796c1df5d166196c0d1aa  
`Testnet Bitcoin Address`: mgMM1SPcCNQW9b3jPJmZkePWx9kar6waFJ  
`Private Key (wif)`: 91bTreuLx5aRX4FhfxFpRkkB9a4zXMQw6qpst5vELmqq2dLTTMv  
## Address 2:
`Private Key (hex)`: 428a14aae29004b707fe3ae6630c79595c5348cbfc23be8becd5a51ba9b869fb  
`Public Key (hex)`: 043314365a064a88759c0dff88b4dda08ec243936dd08b10b98e5974221f47301c7296ac382edc0f54dd854bd06d423142f3d4ff9fba05389adeddf012ca91726d  
`Testnet Bitcoin Address`: n4aAg3a6QuCd8ntxetzKss2hnKLJAWGgCQ  
`Private Key (wif)`: 926DpeVNpQW5gnbph8zCkBwQKC2jSumAK1L8m2tvbtEvbDhbjfb
## Address 3:
`Private Key (hex)`: 448dc43e0e27ab998edb8da347f6e40bdbb5d783f72a257c76e995dcdd3ef625  
`Public Key (hex)`: 041a90a89a2f577df18c246a6fa0410fc1155d9dec19a7ba0b656e70c5d4e4fac893515ec0dbb397978fcce6e7baa361296cad78c62a10f8ed447bc209aa2390c7  
`Testnet Bitcoin Address`: mvBzwrdjU8VcAACm9MQMivHz6aB6xsf91p  
`Private Key (wif)`: 9277H3QbThMiSxEtzxMoaoQjoJ5qhkkMBKqhiWqs3tDPiptEFRB  

2.2: Pay to MultiSig transaction (P2MS)  
Here we recieve more coins from faucet:  
https://live.blockcypher.com/btc-testnet/tx/ee2bf70266b32905d2ba9b86ddf4c76b881d8128068dfcdb9b4398311a6dace4/


In [25]:
def MULTISIG_scriptPubKey(public_key1, public_key2, public_key3):
    return [OP_2, public_key1, public_key2, public_key3, OP_3, OP_CHECKMULTISIG]


def multisigScriptSig(txin, txout, txin_scriptPubKey, private_key1, private_key2):
    signature1 = create_OP_CHECKSIG_signature(txin, [txout], txin_scriptPubKey, private_key1)
    signature2 = create_OP_CHECKSIG_signature(txin, [txout], txin_scriptPubKey, private_key2)
    return [OP_0, signature1, signature2]


def send_from_MULTISIG_transaction(amount, txid_to_spend, utxo_index, txout_scriptPubKey, public_key1, public_key2, public_key3, private_key1, private_key2):
    txout = create_txout(amount, txout_scriptPubKey)
    txin_scriptPubKey = MULTISIG_scriptPubKey(public_key1, public_key2, public_key3)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = multisigScriptSig(txin, txout, txin_scriptPubKey, private_key1, private_key2)
    new_tx = create_signed_transaction(txin, [txout], txin_scriptPubKey, txin_scriptSig)
    return broadcast_transaction(new_tx)

In [26]:
my_private_key = bitcoin.wallet.CBitcoinSecret('92DuwWRXqqF3yxJCvSUvkcjPE2phy2XMS1jGaM8BQMkUfx7dnPk')
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

private_key1 = bitcoin.wallet.CBitcoinSecret('91bTreuLx5aRX4FhfxFpRkkB9a4zXMQw6qpst5vELmqq2dLTTMv')
private_key2 = bitcoin.wallet.CBitcoinSecret('926DpeVNpQW5gnbph8zCkBwQKC2jSumAK1L8m2tvbtEvbDhbjfb')
private_key3 = bitcoin.wallet.CBitcoinSecret('9277H3QbThMiSxEtzxMoaoQjoJ5qhkkMBKqhiWqs3tDPiptEFRB')
public_key1 , public_key2, public_key3 = private_key1.pub, private_key2.pub, private_key3.pub          
address1 = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(public_key1)
address2 = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(public_key2)
address3 = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(public_key3)

multisigScriptPubKey = MULTISIG_scriptPubKey(public_key1, public_key2, public_key3)

txid_to_spend = 'ee2bf70266b32905d2ba9b86ddf4c76b881d8128068dfcdb9b4398311a6dace4'
utxo_index = 0
amount = 0.00010807

response = send_from_P2PKH_transactions([amount], txid_to_spend, utxo_index, [multisigScriptPubKey], my_address, my_private_key, my_public_key)

print("First transaction broadcasted")
print(response.status_code, response.reason)
print(response.text)

First transaction broadcasted
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "68133e81839f5edb08441b2214fe0a48834612e1dfef176ebac57cee96297511",
    "addresses": [
      "zLELbFzx3Hm7YXF269QDZyfRdrPsNg4jcW",
      "moonuSvwZDL5HExuVQuL223RSw63fhJsaL"
    ],
    "total": 10807,
    "fees": 0,
    "size": 399,
    "vsize": 399,
    "preference": "low",
    "relayed_by": "202.61.242.34",
    "received": "2024-05-30T15:12:01.645486009Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "ee2bf70266b32905d2ba9b86ddf4c76b881d8128068dfcdb9b4398311a6dace4",
        "output_index": 0,
        "script": "473044022027a70967e9be2b2ad955b8c8b30cbe8cbee48712482f0f4eb5c0c32907c4b9d5022078ce616ec677b132f41ba11caca7dcda71f93543c17d72d71cdda0d8fb41648d01410470841664631eebe9c1df5972786efef7ab35d5442afd20cf2a85df401c0cb4df8e29ced7be5e5690399b70f6d055ec1b7e6506cd43c67bda097b5a3

In [27]:
txid_to_spend_2 =  "68133e81839f5edb08441b2214fe0a48834612e1dfef176ebac57cee96297511"
utxo_index_2 = 0
txout_scriptPubKey_2 = P2PKH_scriptPubKey(my_address)

response_2 = send_from_MULTISIG_transaction(amount, txid_to_spend_2, utxo_index_2, txout_scriptPubKey_2, public_key1, public_key2, public_key3, private_key1, private_key2)
print("Second transaction broadcasted")
print(response_2.status_code, response_2.reason)
print(response_2.text)

Second transaction broadcasted
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "bc451c6696e394878785256b09f1ab3459890cf97bf11ba9ec919d8bab5969a4",
    "addresses": [
      "zLELbFzx3Hm7YXF269QDZyfRdrPsNg4jcW",
      "moonuSvwZDL5HExuVQuL223RSw63fhJsaL"
    ],
    "total": 10807,
    "fees": 0,
    "size": 231,
    "vsize": 231,
    "preference": "low",
    "relayed_by": "202.61.242.34",
    "received": "2024-05-30T15:13:26.00895485Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "68133e81839f5edb08441b2214fe0a48834612e1dfef176ebac57cee96297511",
        "output_index": 0,
        "script": "00483045022100a1698008779c8366fc22f167e59b296cf49d4c2680f2018aa790c4b0c5beaa5902204d17ce69b90b27895959021ba5156226ce2c37315b63e54d11a5c8a9de2930580147304402205880adfd65a7ac8b3f2b9dddf1746fba66a143936b50f07a894529e3e362f12902203780acea9fb5f7f1e875843901c4b4d5b6144b542

2.3: Custom Transaction (BitCoin Scripting)  
We get some coin again from faucet:  
https://live.blockcypher.com/btc-testnet/tx/e8f80f1108e956726c5ea5c4bb703cc57c978d8b9b8c5eef16f61913c68bd800/

In [28]:
def age_or_pass_scriptPubKey(password):
    return [OP_SUB, OP_GREATERTHANOREQUAL, 18, OP_IF, OP_DROP, OP_TRUE, OP_ELSE, OP_HASH160,
            Hash160(password.to_bytes(4, byteorder='little')), OP_EQUALVERIFY, OP_ENDIF]


def age_or_passScriptSig(password, current_date, birth_date):
    return [password, current_date, birth_date]


def send_from_passScriptSig_transaction(amount, txid_to_spend, utxo_index, txout_scriptPubKey, password, current_date, birth_date):
    txout = create_txout(amount, txout_scriptPubKey)
    txin_scriptPubKey = age_or_pass_scriptPubKey(password)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = age_or_passScriptSig(password, current_date, birth_date)
    new_tx = create_signed_transaction(txin, [txout], txin_scriptPubKey, txin_scriptSig)
    return broadcast_transaction(new_tx)

In [29]:
my_private_key = bitcoin.wallet.CBitcoinSecret('92DuwWRXqqF3yxJCvSUvkcjPE2phy2XMS1jGaM8BQMkUfx7dnPk')
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

amount_to_send = 0.00014509
txid_to_spend = 'e8f80f1108e956726c5ea5c4bb703cc57c978d8b9b8c5eef16f61913c68bd800'
utxo_index = 0

amounts = [0.00014509]

scriptPubKey = age_or_pass_scriptPubKey(810100146)
response = send_from_P2PKH_transactions(amounts, txid_to_spend, utxo_index, [scriptPubKey], my_address, my_private_key, my_public_key)

print("First transaction broadcasted")
print(response.status_code, response.reason)
print(response.text)

First transaction broadcasted
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "31994a206d7cc92fa61211d701c2370250eb286dd46a0c60e542d1fd216a91b5",
    "addresses": [
      "moonuSvwZDL5HExuVQuL223RSw63fhJsaL"
    ],
    "total": 14509,
    "fees": 0,
    "size": 230,
    "vsize": 230,
    "preference": "low",
    "relayed_by": "202.61.242.34",
    "received": "2024-05-30T15:17:22.183859798Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "e8f80f1108e956726c5ea5c4bb703cc57c978d8b9b8c5eef16f61913c68bd800",
        "output_index": 0,
        "script": "47304402206de05c0cc9dba239bb47dc9237c9c0775ad25633c3022fe1a467b64be8b51c75022074b813d3059fe1ed999792d9d8816f12846ecbc3cf3bc0855d4ed2d6691343a701410470841664631eebe9c1df5972786efef7ab35d5442afd20cf2a85df401c0cb4df8e29ced7be5e5690399b70f6d055ec1b7e6506cd43c67bda097b5a3335ef9127",
        "output_value": 14509,
 

In [30]:
txid_to_spend_2 = "31994a206d7cc92fa61211d701c2370250eb286dd46a0c60e542d1fd216a91b5"
utxo_index_2 = 0
txout_scriptPubKey_2 = P2PKH_scriptPubKey(my_address)

response_2 = send_from_passScriptSig_transaction(0.00014509, txid_to_spend_2, utxo_index_2, txout_scriptPubKey_2, 810100146, 2024, 2020)
print("Second transaction broadcasted")
print(response_2.status_code, response_2.reason)
print(response_2.text)

Second transaction broadcasted
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "0505113537cdda3d27ca1e49a76e4a60bfecbb2c5ead41b69daa413baa8a5196",
    "addresses": [
      "moonuSvwZDL5HExuVQuL223RSw63fhJsaL"
    ],
    "total": 14509,
    "fees": 0,
    "size": 96,
    "vsize": 96,
    "preference": "low",
    "relayed_by": "202.61.242.34",
    "received": "2024-05-30T15:18:48.889294557Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "31994a206d7cc92fa61211d701c2370250eb286dd46a0c60e542d1fd216a91b5",
        "output_index": 0,
        "script": "04b225493002e80702e407",
        "output_value": 14509,
        "sequence": 4294967295,
        "script_type": "unknown",
        "age": 0
      }
    ],
    "outputs": [
      {
        "value": 14509,
        "script": "76a9145af03d4c21a6069e0829dfce9bb40ba34168693f88ac",
        "addresses": [
          "moo

Another attempt to see if works the people over 18 years old.  
get money rfom faucet link:  
https://live.blockcypher.com/btc-testnet/tx/56268b5eeba8897971e154814df3d87373fa726877c6f3533a398226d1038b9f/

In [31]:
my_private_key = bitcoin.wallet.CBitcoinSecret('92DuwWRXqqF3yxJCvSUvkcjPE2phy2XMS1jGaM8BQMkUfx7dnPk')
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

amount_to_send = 0.00008926
txid_to_spend = '56268b5eeba8897971e154814df3d87373fa726877c6f3533a398226d1038b9f'
utxo_index = 0

amounts = [0.00008926]

scriptPubKey = age_or_pass_scriptPubKey(810100146)
response = send_from_P2PKH_transactions(amounts, txid_to_spend, utxo_index, [scriptPubKey], my_address, my_private_key, my_public_key)

print("First transaction broadcasted")
print(response.status_code, response.reason)
print(response.text)

First transaction broadcasted
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "1d073be33fe5adda9d37afe0ea5c562863696f88df25934a36042d0b7a18e53a",
    "addresses": [
      "moonuSvwZDL5HExuVQuL223RSw63fhJsaL"
    ],
    "total": 8926,
    "fees": 0,
    "size": 230,
    "vsize": 230,
    "preference": "low",
    "relayed_by": "202.61.242.34",
    "received": "2024-05-30T15:22:14.009372747Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "56268b5eeba8897971e154814df3d87373fa726877c6f3533a398226d1038b9f",
        "output_index": 0,
        "script": "4730440220344b5505bacea4d8534992ea6f4784ea627a7888cae3e58ae5e61a4a8d59770602204bc09781f2957cf4b39c28f25631025816cab584fe5f80e00ffd35e94d467b8e01410470841664631eebe9c1df5972786efef7ab35d5442afd20cf2a85df401c0cb4df8e29ced7be5e5690399b70f6d055ec1b7e6506cd43c67bda097b5a3335ef9127",
        "output_value": 8926,
   

In [32]:
txid_to_spend_2 = "1d073be33fe5adda9d37afe0ea5c562863696f88df25934a36042d0b7a18e53a"
utxo_index_2 = 0
txout_scriptPubKey_2 = P2PKH_scriptPubKey(my_address)

response_2 = send_from_passScriptSig_transaction(0.00008926, txid_to_spend_2, utxo_index_2, txout_scriptPubKey_2, 0, 2024, 2000)
print("Second transaction broadcasted")
print(response_2.status_code, response_2.reason)
print(response_2.text)

Second transaction broadcasted
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "63fdc7b1e12efd27088ac9dba57ff868b0d1b6563902a4713be80b025bae5ec6",
    "addresses": [
      "moonuSvwZDL5HExuVQuL223RSw63fhJsaL"
    ],
    "total": 8926,
    "fees": 0,
    "size": 92,
    "vsize": 92,
    "preference": "low",
    "relayed_by": "202.61.242.34",
    "received": "2024-05-30T15:24:06.725485041Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "1d073be33fe5adda9d37afe0ea5c562863696f88df25934a36042d0b7a18e53a",
        "output_index": 0,
        "script": "0002e80702d007",
        "output_value": 8926,
        "sequence": 4294967295,
        "script_type": "unknown",
        "age": 0
      }
    ],
    "outputs": [
      {
        "value": 8926,
        "script": "76a9145af03d4c21a6069e0829dfce9bb40ba34168693f88ac",
        "addresses": [
          "moonuSvwZDL5HE